In [1]:
#import relevant libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import powerlaw

 

# Set the random seed for data generation (just for test cases)

np.random.seed(42)

In [2]:
import save_and_load_spike_data as spike_data

In [124]:
coupling_r='1meg'

In [125]:
path = spike_data.SAVE_PATH + spike_data.FOLDER + 'fireflys100_'+coupling_r+'OhmCouplingResistance' + spike_data.END
data= spike_data.load_spike_data(path)


In [126]:
print(path)

./spike_data/3fold/fireflys100_1megOhmCouplingResistance.npy


In [127]:
number_of_oscillators=len(data)

In [128]:
#set frame duration -> in the paper this ranged from 1 to 16ms
frame_duration=[0.1,0.064,0.032,0.016,0.008,0.004,0.002,0.001,0.0001,0.00001,0.000001]#50ms
t_simulation=180
Rs=[]
avalanche_data=[]
for f in frame_duration:

  frames = np.linspace(0,t_simulation,int(t_simulation/f)+1)

  spiking_neurons_per_frame=np.zeros(len(frames))
  for i in range(len(data)):
    for j in np.digitize(data[i],frames):
      spiking_neurons_per_frame[j-1]+=1

  avalanche_sizes=[]
  curr_avalanche=False
  for elem in spiking_neurons_per_frame:
    if elem>0:
      if curr_avalanche==False:
        avalanche_sizes.append(elem)
        curr_avalanche=True
      else:
        avalanche_sizes[-1]+=elem
    

    else:
      curr_avalanche=False


  avalanche_data.append(avalanche_sizes)
  results = powerlaw.Fit(avalanche_sizes, discrete=True)
  print(results.power_law.alpha)
  print(results.power_law.xmin)
  R, p = results.distribution_compare('power_law', 'exponential', normalized_ratio=True)#'truncated_power_law', 'lognormal'
  Rse, p = results.distribution_compare('power_law', 'stretched_exponential', normalized_ratio=True)#'truncated_power_law', 'lognormal'
  Rl, p = results.distribution_compare('power_law', 'lognormal', normalized_ratio=True)#'truncated_power_law', 'lognormal'
  Rp, p = results.distribution_compare('power_law', 'lognormal_positive', normalized_ratio=True)#'truncated_power_law', 'lognormal'
  #Rt, p = results.distribution_compare('power_law', 'truncated_power_law', normalized_ratio=True)#'truncated_power_law', 'lognormal'
  print(f"R: {R}, p: {p}")
  Rs.append((int(R),int(Rse),int(Rl),int(Rp)))

  #plt.figure()
  #powerlaw.plot_ccdf(avalanche_sizes, color='r', label=f'R: {R:.1f}, $Δ$t: {1000*f}')
  #plt.legend()
  #plt.xlabel('avalanche sizes')
  #plt.ylabel('ccdf')
  #plt.title('20 all-to-all connected Oscillators over 90s with 0.001k coupling resistance')
  #plt.savefig(f'outputs/fireflies20_ccdf_1_{1000*f}ms.png')

Calculating best minimal value for power law fit
1.457634191277573%
3.0
R: -3.403409033143339, p: 7.834033084366729e-18
Calculating best minimal value for power law fit
1.5782273401449962
3.0
R: 6.643999440460333, p: 4.5755957987721e-08
Calculating best minimal value for power law fit
1.644520096312565%
3.0
R: 12.170042684196464, p: 0.12399196085958432
Calculating best minimal value for power law fit
1.6905431634412582
3.0
R: 15.917855003984531, p: 0.0414744221762983
Calculating best minimal value for power law fit
1.4778097036231457
1.0
R: 27.88676145263536, p: 2.2616197936513583e-21
Calculating best minimal value for power law fit
1.5563644282718956
1.0
R: 37.2788869873819, p: 0.7091576702482918
Calculating best minimal value for power law fit
1.5870269552998084
1.0
R: 40.07510496598935, p: 7.148908838869852e-06
Calculating best minimal value for power law fit
1.809931683026043%
1.0


C:\Users\h3nr1\PycharmProjects\edgeofchaos\venv\lib\site-packages\powerlaw.py:1570: RuntimeWarning: invalid value encountered in true_divide
  return likelihoods/norm


R: 47.291761946573374, p: 1.1428566854828856e-125
Calculating best minimal value for power law fit
1.887925914801226%
2.0
R: 13.577614615400659, p: 0.4827797465834158
Calculating best minimal value for power law fit
3.854163135584488%
2.0
R: -13.975421510132586, p: 0.0
Calculating best minimal value for power law fit
2.428235039047293%
1.0
R: -135.8940529290524, p: 0.0


In [129]:
Rs

[(-3, -8, -8, -8),
 (6, -5, -5, -5),
 (12, -3, -3, -1),
 (15, -1, -2, 2),
 (27, -12, -9, -9),
 (37, -5, -4, 0),
 (40, -2, -2, 4),
 (47, -1, -6, 23),
 (13, -6, -6, 0),
 (-13, -58, -58, -58),
 (-135, -136, -136, -136)]

In [130]:
#rs = pd.DataFrame({'1':Rs}, index=['$Δ$t: 100.0ms','$Δ$t: 64.0ms','$Δ$t: 32.0ms','$Δ$t: 16.0ms','$Δ$t: 8.0ms','$Δ$t: 4.0ms','$Δ$t: 2.0ms','$Δ$t: 1.0ms','$Δ$t: 0.1ms','$Δ$t: 0.01ms','$Δ$t: 0.001ms'])

In [131]:
#rs.head(10)

In [137]:
r_df=pd.read_csv('R_values_fireflies100_180s_3fold.csv',index_col=0)

In [138]:
r_df.head(11)

,1,1k,2500,5k,7500,10k,12500,15k,17500,20k,1meg
$Δ$t: 100.0ms,"(1, 0, 0, 0)","(2, 0, 0, 0)","(0, 0, 0, 0)","(4, 4, 0, 3)","(-8, -9, -10, -10)","(-8, -10, -11, -11)","(-8, -10, -11, -11)","(-5, -7, -8, -8)","(-4, -7, -8, -8)","(-4, -7, -8, -8)","(-3, -8, -8, -8)"
$Δ$t: 64.0ms,"(4, 0, 0, 0)","(0, 0, 0, 0)","(1, 0, 0, 0)","(6, -7, -7, -7)","(2, -8, -8, -8)","(7, -11, -10, -10)","(5, -12, -11, -11)","(3, -8, -7, -7)","(3, -8, -7, -7)","(3, -8, -7, -7)","(6, -5, -5, -5)"
$Δ$t: 32.0ms,"(1, -1, 0, 0)","(0, 0, 0, 0)","(0, -7, -7, -7)","(21, -10, -9, -8)","(15, -11, -10, -10)","(16, -10, -9, -9)","(14, -11, -10, -10)","(9, -12, -11, -11)","(9, -12, -11, -11)","(9, -12, -11, -11)","(12, -3, -3, -1)"
$Δ$t: 16.0ms,"(1, 0, 0, 0)","(0, -1, 0, 0)","(13, -7, -7, -5)","(36, -6, -5, 6)","(28, -10, -9, -1)","(27, -11, -9, -1)","(27, -11, -9, -1)","(21, -10, -9, -6)","(21, -10, -9, -6)","(21, -10, -9, -6)","(15, -1, -2, 2)"
$Δ$t: 8.0ms,"(1, 0, 0, 0)","(-1, -2, -2, -2)","(20, -2, -5, 5)","(44, -1, -3, 14)","(39, -3, -3, 6)","(40, -3, -3, 6)","(38, -4, -4, 2)","(34, -8, -6, -1)","(34, -8, -6, -2)","(34, -8, -6, -2)","(27, -12, -9, -9)"
$Δ$t: 4.0ms,"(0, -1, -1, -1)","(1, 0, -1, 0)","(22, 0, -3, 7)","(45, -1, -4, 18)","(44, -2, -4, 15)","(45, -2, -3, 14)","(42, -1, -1, 6)","(40, -2, -2, 4)","(39, -3, -3, 3)","(39, -3, -3, 3)","(37, -5, -4, 0)"
$Δ$t: 2.0ms,"(0, 0, -1, 0)","(0, -4, -4, -4)","(22, 0, -3, 8)","(46, -1, -7, 23)","(46, -2, -6, 20)","(47, -2, -5, 19)","(42, -1, -1, 8)","(41, -2, -2, 6)","(40, -2, -2, 5)","(40, -2, -2, 5)","(40, -2, -2, 4)"
$Δ$t: 1.0ms,"(0, 0, 0, 0)","(0, -3, -3, -3)","(19, -1, -5, 5)","(47, -4, -11, 33)","(48, -2, -9, 30)","(48, -3, -9, 30)","(47, -2, -8, 26)","(47, -2, -8, 26)","(47, -2, -7, 24)","(47, -1, -7, 25)","(47, -1, -6, 23)"
$Δ$t: 0.1ms,"(0, -8, -8, -8)","(-3, -2, -2, -2)","(3, -2, -2, 0)","(11, -8, -7, -2)","(13, -7, -7, 0)","(4, -3, -3, -1)","(4, -3, -3, -1)","(10, -7, -7, -3)","(9, -7, -7, -4)","(9, -7, -7, -4)","(13, -6, -6, 0)"
$Δ$t: 0.01ms,"(-10, -10, -10, -10)","(-9, -9, -9, -9)","(0, -2, -2, -2)","(-11, -11, -11, -11)","(-10, -10, -10, -10)","(-10, -12, -12, -12)","(-10, -16, -16, -16)","(-10, -18, -18, -18)","(-10, -17, -17, -17)","(-10, -18, -18, -18)","(-13, -58, -58, -58)"


In [134]:
r_df[coupling_r]=Rs

In [135]:
r_df.to_csv('R_values_fireflies100_180s_3fold.csv')

In [136]:
#rs.to_csv('R_values_fireflies100_180s_3fold.csv')